In [18]:
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import numpy as np
import time
from keras.preprocessing.image import save_img

## Download the datset. 
### The orginal dataset can be downloaded from https://github.com/Horea94/Fruit-Images-Dataset
### Reference: Horea Muresan, Mihai Oltean, Fruit recognition from images using deep learning, Acta Univ. Sapientiae, Informatica Vol. 10, Issue 1, pp. 26-42, 2018.

In [ ]:
!wget https://www.dropbox.com/s/l1525goi53teden/fruits-360.zip?dl=0
!mv fruits-360.zip\?dl\=0 fruits-360.zip
!unzip fruits-360.zip
!rm fruits-360.zip

In [2]:
# dimensions of our images.
img_width, img_height = 224, 224 

In [3]:
train_data_dir = './train/'
validation_data_dir = './valid/'
nb_train_samples = 31688
nb_validation_samples = 10657
batch_size = 64

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 33125 images belonging to 81 classes.
Found 8197 images belonging to 81 classes.


### Alternative way to to split train data folder into train and validation is given below.
### This is useful when you just have two folders for Train and Test. 

In [ ]:
# total_datagen = ImageDataGenerator(
#     rescale=1. / 255,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     validation_split=0.2)

# test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
# train_generator = total_datagen.flow_from_directory(
#     train_data_dir,
#     target_size=(img_height, img_width),
#     batch_size=batch_size,
#     class_mode='categorical',
#     subset="training")

# validation_generator = total_datagen.flow_from_directory(
#     validation_data_dir,
#     target_size=(img_height, img_width),
#     batch_size=batch_size,
#     class_mode='categorical',
#      subset="validation")

## Create the ResNet50 Model for transfer learning

In [6]:
inception_base = applications.ResNet50(weights='imagenet', include_top=False)

/anaconda/envs/py35/lib/python3.5/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


### We load the pre-trained ResNet50 network from disk. Do notice how we have
### included the parameter include_top=False – supplying this value indicates 
### that the final fully- connected layers should not be included in the architecture. 
### Therefore, when forward propagating an image through the network, we’ll obtain the
### feature values after the final POOL layer rather than the probabilities produced by 
### the softmax classifier in the FC layers.

In [7]:
x = inception_base.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(81, activation='softmax')(x)
inception_transfer = Model(inputs=inception_base.input, outputs=predictions)

In [8]:
inception_transfer.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [9]:
inception_transfer.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [ ]:
inception_transfer.fit_generator(
    train_generator,
    steps_per_epoch=33125 // 64,
    epochs=5, shuffle = True, verbose = 1, 
    max_queue_size=10,
    validation_data=validation_generator,
	validation_steps=8197 // 64)

## What does the Network see?

In [10]:
def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [13]:
def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + K.epsilon())

In [12]:
input_img = inception_transfer.input
layer_dict = dict([(layer.name, layer) for layer in inception_transfer.layers[1:]])


In [14]:
layer_name = 'res5b_branch2a'

In [19]:
kept_filters = []
for filter_index in range(200):
    # we only scan through the first 200 filters,
    # but there are actually 512 of them
    print('Processing filter %d' % filter_index)
    start_time = time.time()

    # we build a loss function that maximizes the activation
    # of the nth filter of the layer considered
    layer_output = layer_dict[layer_name].output
    if K.image_data_format() == 'channels_first':
        loss = K.mean(layer_output[:, filter_index, :, :])
    else:
        loss = K.mean(layer_output[:, :, :, filter_index])

    # we compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, input_img)[0]

    # normalization trick: we normalize the gradient
    grads = normalize(grads)

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_img], [loss, grads])

    # step size for gradient ascent
    step = 1.

    # we start from a gray image with some random noise
    if K.image_data_format() == 'channels_first':
        input_img_data = np.random.random((1, 3, img_width, img_height))
    else:
        input_img_data = np.random.random((1, img_width, img_height, 3))
    input_img_data = (input_img_data - 0.5) * 20 + 128

    # we run gradient ascent for 20 steps
    for i in range(20):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step

        print('Current loss value:', loss_value)
        if loss_value <= 0.:
            # some filters get stuck to 0, we can skip them
            break

    # decode the resulting input image
    if loss_value > 0:
        img = deprocess_image(input_img_data[0])
        kept_filters.append((img, loss_value))
    end_time = time.time()
    print('Filter %d processed in %ds' % (filter_index, end_time - start_time))


Processing filter 0
Current loss value: -1.8854146
Filter 0 processed in 4s
Processing filter 1
Current loss value: -0.3695934
Filter 1 processed in 2s
Processing filter 2
Current loss value: -0.54007083
Filter 2 processed in 3s
Processing filter 3
Current loss value: -1.092056
Filter 3 processed in 3s
Processing filter 4
Current loss value: 0.45698902
Current loss value: 1.2760646
Current loss value: 2.3000803
Current loss value: 3.2999067
Current loss value: 4.3154783
Current loss value: 4.864712
Current loss value: 5.586628
Current loss value: 6.071331
Current loss value: 6.840914
Current loss value: 7.493054
Current loss value: 8.102612
Current loss value: 8.67634
Current loss value: 9.129922
Current loss value: 9.755965
Current loss value: 10.142498
Current loss value: 10.8106
Current loss value: 11.057241
Current loss value: 11.730542
Current loss value: 11.846345
Current loss value: 12.382079
Filter 4 processed in 5s
Processing filter 5
Current loss value: -0.026552707
Filter 5 

In [24]:
# we will stich the best 64 filters on a 8 x 8 grid.
n = 5

# the filters that have the highest loss are assumed to be better-looking.
# we will only keep the top 64 filters.
kept_filters.sort(key=lambda x: x[1], reverse=True)
kept_filters = kept_filters[:n * n]

# build a black picture with enough space for
# our 8 x 8 filters of size 128 x 128, with a 5px margin in between
margin = 5
width = n * img_width + (n - 1) * margin
height = n * img_height + (n - 1) * margin
stitched_filters = np.zeros((width, height, 3))

# fill the picture with our saved filters
for i in range(n):
    for j in range(n):
        print(i,j)
        img, loss = kept_filters[i * n + j]
        width_margin = (img_width + margin) * i
        height_margin = (img_height + margin) * j
        stitched_filters[
            width_margin: width_margin + img_width,
            height_margin: height_margin + img_height, :] = img

# save the result to disk
save_img('stitched_filters_%dx%d.png' % (n, n), stitched_filters)

0 0
0 1
0 2
0 3
0 4
1 0
1 1
1 2
1 3
1 4
2 0
2 1
2 2
2 3
2 4
3 0
3 1
3 2
3 3
3 4
4 0
4 1
4 2
4 3
4 4


In [25]:
??applications.ResNet50

Signature: applications.ResNet50(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
Source:   
def ResNet50(include_top=True,
             weights='imagenet',
             input_tensor=None,
             input_shape=None,
             pooling=None,
             classes=1000):
    """Instantiates the ResNet50 architecture.

    Optionally loads weights pre-trained on ImageNet.
    Note that the data format convention used by the model is
    the one specified in your Keras config at `~/.keras/keras.json`.

    # Arguments
        include_top: whether to include the fully-connected
            layer at the top of the network.
        weights: one of `None` (random initialization),
              'imagenet' (pre-training on ImageNet),
              or the path to the weights file to be loaded.
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: